In [ ]:
%pip install pandas

In [ ]:
from utils_json_xlxs.xlsx_extractor import *

from utils_json_xlxs.extractor_config import ExtractorConfig
from utils_json_xlxs.jsons_to_df import (
    cdc_json_to_df,
    cv_json_single_file_to_df,
    rec_json_to_df,
)

In [ ]:
ao_config_toml_path = "/workspace/notebooks/json/2023DOS0550696/extractor_config.toml"
conf = ExtractorConfig(ao_config_toml_path)
extractor = XlsxExtractor(conf)
extractor.save_to_xlsx()

# ROADMAP

refac code [ ]  
  
être le plus générique possible en terme de format et/ou pouvoir faire des modifs facilement (fonction dédiée) [ ]  
  
/src/dag_nao/outputs/2023..696/entreprise/datetime/  
-> les jsons et l'excel (+ document d'observabilité sur le prompt ?)
  
automatiser la génération (soit en auomatisant création toml soit sans passer par toml) [ ]  

# PIPE

- genere json ao (rec, cdc) et cv
- convert json ao to sheet 1
- convert json cv to sheet 2

## AO

### REC

In [ ]:
rec_json_path = "/workspace/notebooks/json/2023DOS0550696/amaris/rec1.json"

rec_df = rec_json_to_df(rec_json_path)

In [ ]:
rec_df.head()

In [ ]:
rec_df["index"].values.tolist()

In [ ]:
rec_df["data"].values.tolist()

### CDC

In [ ]:
cdc_json_path = "/workspace/notebooks/json/2023DOS0550696/amaris/cdc1.json"

cdc_df = cdc_json_to_df(cdc_json_path)

cdc_df.head()

In [ ]:
cdc_df["index"].values.tolist()

In [ ]:
cdc_df["data"].values.tolist()

### CDC + REC

In [ ]:
ao_df = XlsxExtractor._generate_ao_df(rec_json_path, cdc_json_path, has_metadata=True)

ao_df.head()

In [ ]:
ao_df["index"].values.tolist()

## AO REFAC

In [ ]:
def json_to_df_ao(rec_path: str,
                    cdc_path: str,
                    has_metadata: bool) -> pd.DataFrame:
    """Generates the SNCF AO dataframe from the SNCF JSONs and returns it

    Parameters:
        rec_path (str): path to the SNCF JSON
        cdc_path (str): path to the CDC JSON

    Returns:
        oa_df (pd.DataFrame): AO dataframe
    """
    # fetch raw dataframes from SNCF and concatenate
    rec_df = rec_json_to_df(rec_path)
    cdc_df = cdc_json_to_df(cdc_path)
    
    oa_df = pd.concat([rec_df, cdc_df], ignore_index=True)

    # split metadata and actual data in separate colums
    if has_metadata:
        oa_df = split_metadata(oa_df)
    return oa_df

In [ ]:
def json_to_xlsx_ao(config: ExtractorConfig) -> None:
    """Writes JSON data to xlsx sheet

    Generate the ao_dataframe, then the cv_dataframe for each profile and write to xlsx 
    sheet.
    This class need no argument as every path is defined in the config object.
    """
    # Writer config
    writer = pd.ExcelWriter(config.XLSX_OUTNAME, engine="xlsxwriter")
    workbook = writer.book
    formats = workbook.add_format({"text_wrap": True})

    # Generate SNCF AO dataframe and write to xlsx sheet 1
    ao_df = json_to_df_ao(config.REC_JSON_NAME,
                        config.CDC_JSON_NAME,
                        config.HAS_METADATA)
    sheet_name = "ao_sncf"
    ao_df.to_excel(writer, sheet_name=sheet_name, index=False)
    for column in ao_df:
        col_idx = ao_df.columns.get_loc(column)
        if column == "data":
            writer.sheets[sheet_name].set_column(col_idx, col_idx, 60, formats)
        else:
            writer.sheets[sheet_name].set_column(col_idx, col_idx, 30, formats)
    writer.close()

In [ ]:
ao_config_toml_path = "/workspace/notebooks/json/2023DOS0550696/extractor_config.toml"
conf = ExtractorConfig(ao_config_toml_path)

json_to_df_ao(rec_path=conf.REC_JSON_NAME, cdc_path=conf.CDC_JSON_NAME, has_metadata=True)

In [ ]:
json_to_xlsx_ao(conf)

## OFFRE TECH

In [ ]:
cv_json_path = "/workspace/notebooks/json/2023DOS0550696/amaris/cv1.json"

cv_df = XlsxExtractor._generate_cv_df(cv_json_path, True)
cv_df

## OFFRE TECH REFAC

In [ ]:
def generate_cv_df(paths_to_json: str,
                    has_metadata: bool) -> pd.DataFrame:
    """Generates the CV dataframe from the CV JSONs (experiences, taches, diplomes)

    Some cleaning is done in this function:
    - replace NaNs by list containing only the N/A value.
    - flatten nested lists
    - split metadata and actual data in separate colums

    Parameters:
        paths_to_json (str): path to the CV JSON
    Returns:
        cv_df (pd.DataFrame): CV dataframe
    """
    # fetch raw dataframes
    # cv_exp_df = cv_exp_json_to_df(exp_path)
    # cv_tasks_df = cv_tasks_json_to_df(tasks_path)
    # cv_diplomae_df = cv_diplomae_json_to_df(diplomae_path)
    cv_exp_df, cv_tasks_df, cv_diplomae_df = cv_json_single_file_to_df(paths_to_json)

    # prepare the concatenation dataframe
    cv_df = pd.DataFrame(columns=["categorie", "index", "data"])

    entreprises = cv_exp_df["categorie"].unique()

    for e in entreprises:
        df_1 = cv_exp_df[cv_exp_df["categorie"] == e]

        df_2 = cv_tasks_df[cv_tasks_df["categorie"] == e]
        df_2 = df_2.drop(df_2[df_2["index"] == "nom_entreprise"].index)
        df_2 = df_2.drop(df_2[df_2["index"] == "nom_poste"].index)

        df_1 = pd.concat([df_1, df_2], ignore_index=True)

        cv_df = pd.concat([cv_df, df_1], ignore_index=True)

    cv_df = pd.concat([cv_df, cv_diplomae_df], ignore_index=True)

    # split the metadata into separate columns
    if has_metadata:
        cv_df = split_metadata(cv_df)

    return cv_df

In [ ]:
def cv_json_single_file_to_df(filename: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Reads the CV JSON output and returns a dataframe"""
    data = _load_json(filename)

    # experience
    cv_exp_df = pd.DataFrame(columns=["categorie", "index", "data"])
    for xp in data["experiences_entreprise"]:
        tmp_df = pd.json_normalize(xp).T.reset_index()
        tmp_df.columns = ["index", "data"]

        categorie_name = tmp_df[tmp_df["index"] == "nom_entreprise"]["data"][0][0]
        tmp_df["categorie"] = categorie_name

        cv_exp_df = pd.concat([cv_exp_df, tmp_df], ignore_index=True)

    # tasks
    cv_taches_df = pd.DataFrame(columns=["categorie", "index", "data"])
    for xp in data["experiences_entreprise"]:
        tmp_df = pd.json_normalize(xp).T.reset_index()
        tmp_df.columns = ["index", "data"]

        tmp_dg = tmp_df[tmp_df["index"] == "role"].explode(column="data")
        tmp_df = tmp_df.drop(tmp_df[tmp_df["index"] == "role"].index)
        tmp_df = pd.concat([tmp_df, tmp_dg], ignore_index=True)

        categorie_name = tmp_df[tmp_df["index"] == "nom_entreprise"]["data"][0][0]
        tmp_df["categorie"] = categorie_name

        cv_taches_df = pd.concat([cv_taches_df, tmp_df], ignore_index=True)

    # degrees
    cv_diplomes_df = pd.json_normalize(data).T.reset_index()
    cv_diplomes_df.columns = ["index", "data"]

    cv_diplomes_df = cv_diplomes_df.explode(column="data")
    cv_diplomes_df["categorie"] = "Divers (diplomes, ...)"
    cv_diplomes_df = cv_diplomes_df[["categorie", "index", "data"]]
    cv_diplomes_df = cv_diplomes_df[cv_diplomes_df["index"] != "experiences_entreprise"]

    return cv_exp_df, cv_taches_df, cv_diplomes_df

In [ ]:
import json
cv_path = "/workspace/notebooks/json/2023DOS0550696/amaris/cv1.json"
with open(cv_path, "rb") as f:
    data = json.load(f)

In [ ]:
list(data.keys())

In [ ]:
list(data.values())[0]

In [ ]:
v = list(data.values())
print(len(v))
v[-1]

In [ ]:
t = ['SAP Smart', {'page': 3}]
t[-1].keys()

In [ ]:
from typing import Dict
def elementary_list_check(L:list)->bool:
    if len(L)==2:
        if isinstance(L[-1],Dict):
            if 'page' in L[-1].keys():
                return True
    return False

In [ ]:
def split_elementary_list(L:list):
    return L[0], L[-1]['page']

In [ ]:
dict_test = v[-1][0]
dict_test

In [ ]:
data["diplomes"]

In [ ]:
for key in dict_test.keys():
    print(key)
    val = dict_test[key]
    if elementary_list_check(val):
        info, page = split_elementary_list(val)
    else:
        for sub in val:
            if elementary_list_check(sub):
                info, page = split_elementary_list(sub)
                print(sub)

In [ ]:
def write_property_in_sheet(D:dict):
    for key in D.keys():
        print(key)
        val = dict_test[key]
        if elementary_list_check(val):
            info, page = split_elementary_list(val)
        else:
            for sub in val:
                if elementary_list_check(sub):
                    info, page = split_elementary_list(val)

In [ ]:
t = ['SAP Smart', {'page': 3}]
print(elementary_list_check(t))
split_elementary_list(t)

In [ ]:
%pip install xlsxwriter

In [ ]:
def write_property_info_in_excel(worksheet, data, property_name, metadata=False):
    row, col = 0, 0

    for dict in data[property_name]:
            
            for key in dict:
                # column 0 : name of element
                worksheet.write(row, col, key)

                val = dict[key]
                if metadata:
                    if elementary_list_check(val):
                        info, page = split_elementary_list(val)
                        worksheet.write(row, col+1, info)
                        worksheet.write(row, col+2, page)
                        row += 1

                    else:
                        for sub in val:
                            if elementary_list_check(sub):
                                info, page = split_elementary_list(sub)
                                
                                worksheet.write(row, col+1, info)
                                worksheet.write(row, col+2, page)
                                row += 1
                
                else:
                    if not isinstance(val, list):
                        worksheet.write(row, col+1, val)
                        row += 1
                    else:
                        for sub in val:
                            if not isinstance(val, Dict):
                                worksheet.write(row, col+1, sub)
                                row += 1
                            else


                        
        row += 5

In [ ]:
if isinstance([2,4], list):
    print("hello")

In [ ]:
import xlsxwriter

# open excel file
out = "/workspace/notebooks/outputs/hello.xlsx"
workbook = xlsxwriter.Workbook(out)

# read json
cv_path = "/workspace/notebooks/json/2023DOS0550696/amaris/cv1.json"
with open(cv_path, 'r') as file:
    data = json.load(file)

# keys of json ['diplomes', 'certifications', 'langues_parlees', 'experiences_entreprise']
# properties_names = list(data.keys())


for property_name in data.keys():
    # create sheet
    worksheet = workbook.add_worksheet(property_name)
    # reset row and column indexes
    row, col = 0, 0
    
    for dict in data[property_name]:
        for key in dict:
            print(key)
            worksheet.write(row, col, key)

            val = dict[key]
            if elementary_list_check(val):
                info, page = split_elementary_list(val)
                worksheet.write(row, col+1, info)
                worksheet.write(row, col+2, page)
                row += 1

            else:
                for sub in val:
                    if elementary_list_check(sub):
                        info, page = split_elementary_list(sub)
                        
                        worksheet.write(row, col+1, info)
                        worksheet.write(row, col+2, page)
                        row += 1
        
        row += 5



workbook.close()

### claude 3

In [ ]:
%pip install xlsxwriter

In [ ]:
import json
cv_path = "/workspace/notebooks/json/2022DOS1246222/amaris/formatted_json_cv.json"
with open(cv_path, "rb") as f:
    data = json.load(f)

data

In [ ]:
from typing import List, Dict

In [ ]:
experiences = data["experiences_entreprise"]
diplomes = data["diplomes"]

In [ ]:
for exp in diplomes:
    for key in exp.keys():
        print(key)
        print(type(key))
        val = exp[key]
        if not isinstance(val, List):
            print(val)
        else:
            for sub in val:
                print(sub)

In [ ]:
## REPONSE AO

import json
import xlsxwriter



num_dos = "2023DOS0550696"
entreprise = "neurones_JMB"

# open excel file
out = f"/workspace/notebooks/outputs/{num_dos}/{entreprise}/reponse_ao.xlsx"
workbook = xlsxwriter.Workbook(out)

# read json
cv_path = f"/workspace/notebooks/json/{num_dos}/{entreprise}/formatted_json_cv.json"
with open(cv_path, 'r') as file:
    data = json.load(file)

# keys of json ['diplomes', 'certifications', 'langues_parlees', 'experiences_entreprise']
# properties_names = list(data.keys())


for property_name in data.keys():
    # create sheet
    worksheet = workbook.add_worksheet(property_name)
    # reset row and column indexes
    row, col = 0, 0
    
    for elmt in data[property_name]:
        for key in elmt.keys():
            print(key)
            worksheet.write(row, col, key)

            val = elmt[key]
            if not isinstance(val, List):
                print(val)
                worksheet.write(row, col+1, val)
                row += 1

            else:
                for sub in val:
                    print(sub)
                    worksheet.write(row, col+1, sub)
                    row += 1
        
        row += 4

# read json bpu
bpu_path = f"/workspace/notebooks/json/{num_dos}/{entreprise}/formatted_json_bpu.json"
with open(bpu_path, 'r') as file:
    data = json.load(file)

worksheet = workbook.add_worksheet("offre financiere")
row, col = 0, 0
for k in data.keys():
    
    worksheet.write(row, col, k)
    worksheet.write(row, col+1, data[k])
    row+=1

workbook.close()

In [3]:
import xlsxwriter
out = "/workspace/notebooks/a.xlsx"
workbook = xlsxwriter.Workbook(out)
worksheet = workbook.add_worksheet("offre financiere")
type(worksheet)

xlsxwriter.worksheet.Worksheet

In [4]:
def write_property_in_sheet(ws:xlsxwriter.worksheet.Worksheet, property_data:Dict, row:int, col:int):
    for elmt_name in property_data.keys():
        elmt_val = property_data[elmt_name]
        ws.write(row, col, elmt_name)
        
        if not isinstance(elmt_val, List):
                ws.write(row, col+1, elmt_val)
                row += 1
                
        else:
            for sub_elmt in elmt_val:
                if isinstance(sub_elmt, Dict):
                    c = 1
                    for k in sub_elmt.keys():
                        ws.write(row, col+c, sub_elmt[k])
                        c += 1
                    
                else:
                    ws.write(row, col+c, sub_elmt)
                row += 1

NameError: name 'Dict' is not defined

In [ ]:
def write_bpu_in_workbook(wb:xlsxwriter.workbook.Workbook, bpu_data:Dict):
    worksheet = wb.add_worksheet("offre financiere")
    row, col = 0, 0
    for k in bpu_data.keys():
        worksheet.write(row, col, k)
        worksheet.write(row, col+1, bpu_data[k])
        row+=1

In [ ]:
# AO

num_dos = "2023DOS0550696"
entreprise = "neurones_JMB"

# open excel file
out = f"/workspace/notebooks/outputs/{num_dos}/{entreprise}/ao.xlsx"
workbook = xlsxwriter.Workbook(out)

# read json
rec_path = f"/workspace/notebooks/json/{num_dos}/{entreprise}/formatted_json_rec.json"
with open(rec_path, 'r') as file:
    data = json.load(file)

# keys of json ['diplomes', 'certifications', 'langues_parlees', 'experiences_entreprise']
# properties_names = list(data.keys())
worksheet = workbook.add_worksheet("REC et CDC")
row, col = 0, 0
for elmt_name in data.keys():
    
    elmt_val = data[elmt_name]
    worksheet.write(row, col, elmt_name)
    
    if not isinstance(elmt_val, List):
            worksheet.write(row, col+1, elmt_val)
            row += 1
            
    else:
        for sub_elmt in elmt_val:
            if isinstance(sub_elmt, Dict):
                c = 1
                for k in sub_elmt.keys():
                    worksheet.write(row, col+c, sub_elmt[k])
                    c += 1
                
            else:
                worksheet.write(row, col+c, sub_elmt)
            row += 1

row += 4

# read json cdc
cdc_path = f"/workspace/notebooks/json/{num_dos}/{entreprise}/formatted_json_cdc.json"
with open(cdc_path, 'r') as file:
    data = json.load(file)

for elmt_name in data.keys():
    
    elmt_val = data[elmt_name]
    worksheet.write(row, col, elmt_name)
    
    if not isinstance(elmt_val, List):
            worksheet.write(row, col+1, elmt_val)
            row += 1
            
    else:
        for sub_elmt in elmt_val:
            if isinstance(sub_elmt, Dict):
                c = 1
                for k in sub_elmt.keys():
                    worksheet.write(row, col+c, sub_elmt[k])
                    c += 1
                
            else:
                worksheet.write(row, col+1, sub_elmt)
            row += 1

workbook.close()